## GAN : Generative Adversarial Networks

##### Practice thesis code

In [2]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

#### 생성자(Generator) 및 판별자(Discriminator)모델 정의

In [22]:
latent_dim=100

# 생성자 클래스 정의
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # 하나의 블록(block)정의
        def block(input_dim, output_dim, normalize=True):
            layers=[nn.Linear(input_dim, output_dim)]
            if normalize:
                # 배치 정규화(batch normaliztion) 수행(차원 동일)
                layers.append(nn.BatchNorm1d(output_dim, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # 생성자 모델은 연속적인 여러 개의 블록을 가짐
        self.model=nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 1 * 28 * 28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img
        

In [26]:
# 판별자 클래스 정의
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model=nn.Sequential(
            nn.Linear(1*28*28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    # 이미지에 대한 판별 결과를 반환
    def forword(self, img):
        flattened = img.view(img.size(0), -1)
        output = self.model(flattened)

        return output

### 학습 데이터셋 불러오기

##### * MNIST DATA SET

In [24]:
transforms_train=transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

train_dataset=datasets.MNIST(root="./dataset", train=True, download=True, transform=transforms_train)
dataloaders=torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)

### 모델 학습 및 샘플링

##### * 학습을 위해 생성자와 판별자 모델 초기화
##### * 적절한 하이퍼 파라미터 설정

In [28]:
# 생성자와 판별자 초기화
generator = Generator()
discriminator = Discriminator()

generator.cuda()
discriminator.cuda()

# 손실함수 (loss function)
adversarial_loss=nn.BCELoss()
adversarial_loss.cuda()

# 학습률(learning rate)설정
lr=0.0002

# 생성자와 판별자를 위한 최적화 함수
optimizer_G=torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D=torch.optim.Adam(discriminator.paramerters(), lr=lr, betas=(0.5, 0.999))

AssertionError: Torch not compiled with CUDA enabled